In [1]:
import ee
import geemap
from datetime import datetime, timedelta
import math
import matplotlib.pyplot as plt
import numpy as np
import ipynb
import time
from speckle_filter import leesigma

In [3]:
# Get authentication credentials to initialize ee
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AWtgzh6LiOQrdtjbqiQmLkGYK9qC3g89j3Fq-0FwbyQRNGjElEFtAcOBhM8

Successfully saved authorization token.


In [4]:
# Define geometry for data collection

roi = ee.Geometry.Rectangle([69.98568, 22.66953, 72.76651, 24.60271]);  # Nanda Bet S1 extent
roi_small = ee.Geometry.Rectangle([70.7980547394194417, 23.2880385161501735, 71.5060515087354958, 23.9024834537972986])  # Nanda Bet small region


# roi_alaska = ee.Geometry.Rectangle([-161.2807843239999954, 60.1563621669999975, -161.1689946729999861, 60.2120380060000002]) # Region in Alaska for Bruce
# clipping_mask = roi_alaska



# boundary shapefiles
shp_file = 'data/s1_boundary.shp'
s1_boundary = geemap.shp_to_ee(shp_file)
# type(s1_boundary)

shp_file = 'data/smaller_region.shp'
small_boundary = geemap.shp_to_ee(shp_file)

clipping_mask = small_boundary



In [5]:
type(clipping_mask)

ee.featurecollection.FeatureCollection

In [6]:
# check actual date of s2 or dw

d1 = '2020-01-01'
d2 = '2021-01-01'

#s2_collection = ee.ImageCollection('COPERNICUS/S2').filterBounds(s1_boundary).filterDate(d1, d2)  
s2_collection = ee.ImageCollection('COPERNICUS/S2').filterBounds(clipping_mask).filterDate(d1, d2)  

# print number of images in collection
print(s2_collection.size().getInfo())

# print dates of acquisition
acq_times = s2_collection.aggregate_array('system:time_start').getInfo()
print([time.strftime('%m-%d-%Y', time.gmtime(t/1000)) for t in acq_times])


292
['01-01-2020', '01-01-2020', '01-01-2020', '01-01-2020', '01-06-2020', '01-06-2020', '01-06-2020', '01-06-2020', '01-11-2020', '01-11-2020', '01-11-2020', '01-11-2020', '01-16-2020', '01-16-2020', '01-16-2020', '01-16-2020', '01-21-2020', '01-21-2020', '01-21-2020', '01-21-2020', '01-26-2020', '01-26-2020', '01-26-2020', '01-26-2020', '01-31-2020', '01-31-2020', '01-31-2020', '01-31-2020', '02-05-2020', '02-05-2020', '02-05-2020', '02-05-2020', '02-10-2020', '02-10-2020', '02-10-2020', '02-10-2020', '02-15-2020', '02-15-2020', '02-15-2020', '02-15-2020', '02-20-2020', '02-20-2020', '02-20-2020', '02-20-2020', '02-25-2020', '02-25-2020', '02-25-2020', '02-25-2020', '03-01-2020', '03-01-2020', '03-01-2020', '03-01-2020', '03-06-2020', '03-06-2020', '03-06-2020', '03-06-2020', '03-11-2020', '03-11-2020', '03-11-2020', '03-11-2020', '03-16-2020', '03-16-2020', '03-16-2020', '03-16-2020', '03-21-2020', '03-21-2020', '03-21-2020', '03-21-2020', '03-26-2020', '03-26-2020', '03-26-2020', '

In [9]:
# print band names
print(s2_collection.first().bandNames().getInfo())

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'QA10', 'QA20', 'QA60']


In [7]:
# get Dynamic World data
dw_collection = (ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")
    .filterBounds(clipping_mask)
    .filterDate(d1, d2))  

    
# print number of images in collection
print(dw_collection.size().getInfo())

# print dates of acquisition
acq_times = dw_collection.aggregate_array('system:time_start').getInfo()
print([time.strftime('%m-%d-%Y', time.gmtime(t/1000)) for t in acq_times])


217
['01-06-2020', '01-06-2020', '01-06-2020', '01-06-2020', '01-11-2020', '01-11-2020', '01-11-2020', '01-11-2020', '01-16-2020', '01-16-2020', '01-16-2020', '01-16-2020', '01-21-2020', '01-21-2020', '01-21-2020', '01-21-2020', '01-26-2020', '01-26-2020', '01-26-2020', '01-26-2020', '01-31-2020', '01-31-2020', '01-31-2020', '01-31-2020', '02-05-2020', '02-05-2020', '02-05-2020', '02-05-2020', '02-10-2020', '02-10-2020', '02-10-2020', '02-10-2020', '02-15-2020', '02-20-2020', '02-20-2020', '02-20-2020', '02-20-2020', '02-25-2020', '02-25-2020', '02-25-2020', '02-25-2020', '03-01-2020', '03-01-2020', '03-01-2020', '03-01-2020', '03-06-2020', '03-06-2020', '03-06-2020', '03-06-2020', '03-11-2020', '03-11-2020', '03-11-2020', '03-11-2020', '03-21-2020', '03-21-2020', '03-21-2020', '03-21-2020', '03-31-2020', '03-31-2020', '03-31-2020', '03-31-2020', '04-05-2020', '04-05-2020', '04-05-2020', '04-05-2020', '04-10-2020', '04-10-2020', '04-10-2020', '04-10-2020', '04-15-2020', '04-15-2020', '

In [6]:
water = dw_collection.select('water').mean().clip(clipping_mask).rename('water');
label = dw_collection.select('label').mode().clip(clipping_mask).rename('label');
flooded_vegetation = dw_collection.select('flooded_vegetation').mean().clip(clipping_mask).rename('flooded_vegetation');
bare = dw_collection.select('bare').mean().clip(clipping_mask).rename('bare');
trees = dw_collection.select('trees').mean().clip(clipping_mask).rename('trees');
grass = dw_collection.select('grass').mean().clip(clipping_mask).rename('grass');
crops = dw_collection.select('crops').mean().clip(clipping_mask).rename('crops');
shrub_and_scrub = dw_collection.select('shrub_and_scrub').mean().clip(clipping_mask).rename('shrub_and_scrub');
built = dw_collection.select('built').mean().clip(clipping_mask).rename('built');
snow_and_ice = dw_collection.select('snow_and_ice').mean().clip(clipping_mask).rename('snow_and_ice');

dw_img = water.addBands(ee.Image([bare,flooded_vegetation,trees,grass,crops,shrub_and_scrub,built,snow_and_ice]))

In [10]:
# Get image from collection
s2_img = ee.Image(s2_collection.mean().divide(10000).clip(clipping_mask.geometry()))

#s2_img = ee.Image(s2_collection.mean().divide(10000).clip(clipping_mask))

In [11]:
# Disply S2 image on map


# define visualization parameters
s2_vis = {
    'bands': ['B4','B3','B2'],
    'min': 0.0,
    'max': 0.3,
}


# set up map
M = geemap.Map()
M.centerObject(clipping_mask.geometry(),12)
#M.centerObject(clipping_mask,12)
# M.add_basemap('SATELLITE')
M.addLayer(s2_img, s2_vis, 's2')



# display map
M

Map(center=[23.54320534531615, 71.10409144438285], controls=(WidgetControl(options=['position', 'transparent_b…

In [12]:
# print band names
print(s2_img.bandNames().getInfo())

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'QA10', 'QA20', 'QA60']


In [13]:
# Export single S2 image
txt = 's2_nandabet_20200301_all_bands' 
task = ee.batch.Export.image.toDrive(image=s2_img,
                                     description=txt,
                                     scale=20,
                                     fileNamePrefix=txt,
                                     crs='EPSG:4326',
                                     fileFormat='GeoTIFF',
                                     maxPixels=2e10,
                                     region=clipping_mask.geometry())
                                     #region=clipping_mask)
task.start()


In [7]:
# Export single DW image
txt = 'dw_bands_nandabet_20200301_20m'
task = ee.batch.Export.image.toDrive(image=dw_img,
                                     description=txt,
                                     scale=20,
                                     fileNamePrefix=txt,
                                     crs='EPSG:4326',
                                     fileFormat='GeoTIFF',
                                     maxPixels=2e10,
                                     region=clipping_mask.geometry())
                                     #region=clipping_mask)
task.start()


txt = 'dw_label_nandabet_20200301_20m'
task = ee.batch.Export.image.toDrive(image=label,
                                     description=txt,
                                     scale=20,
                                     fileNamePrefix=txt,
                                     crs='EPSG:4326',
                                     fileFormat='GeoTIFF',
                                     maxPixels=2e10,
                                     region=clipping_mask.geometry())
                                     #region=clipping_mask)
task.start()



In [8]:
# Check export status
task.list()

[<Task TQVBNT7GYYIRPAPG4CH4U6A5 EXPORT_IMAGE: s1_nandabet_03-03-2020_20m (COMPLETED)>,
 <Task OWIHZ7CGYJ4US6N2SN2XADKO EXPORT_IMAGE: dw_label_nandabet_20200301_20m (COMPLETED)>,
 <Task EHSTRGBKUSWYUIKY4B5QVZFX EXPORT_IMAGE: dw_bands_nandabet_20200301_20m (RUNNING)>,
 <Task BJGCCBABWUWE37IK5D7VORSB EXPORT_IMAGE: s2_nandabet_20200303_all_bands (COMPLETED)>,
 <Task BGORK4R5DYOO4W7PH2HGWYKW EXPORT_IMAGE: s1_nanda_bet_08-18-2020 (COMPLETED)>,
 <Task XO2LXBBZWYNR6W3ITQF5QMHD EXPORT_IMAGE: s1_everglades_12-06-2021 (COMPLETED)>,
 <Task OPNHMXYRWGANN2IFFQVHRK4Q EXPORT_IMAGE: s1_everglades_12-06-2021 (COMPLETED)>]

In [6]:
# Import s2 and dynamic world data in a loop

dates_beg = ['2020-01-15','2020-01-26','2020-02-07','2020-02-20','2020-03-01','2020-04-07','2020-04-30','2020-05-12',
             '2020-05-25','2020-09-22','2020-10-27','2020-11-08','2020-11-21','2020-12-03']
             
dates_end = ['2020-01-17','2020-01-29','2020-02-11','2020-02-23','2020-03-05','2020-04-14','2020-05-03','2020-05-16',
             '2020-05-28','2020-09-25','2020-10-31','2020-11-12','2020-11-25','2020-12-07']
    

n = len(dates_beg)

task_id = []

for idx in range(n):
    d1 = dates_beg[idx]
    d2 = dates_end[idx]
    
    # get Dynamic World data
    dw_collection = (ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")
        .filterBounds(s1_boundary)
        .filterDate(d1, d2))    

#     # get Sentinel-2 data
#     s2_collection = ee.ImageCollection('COPERNICUS/S2').filterBounds(s1_boundary).filterDate(d1, d2)  

#     # get Sentinel-1 data
#     s1_collection = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
#                      .filter(ee.Filter.eq('instrumentMode', 'IW'))
#                      .filterMetadata('resolution_meters', 'equals', 10)
#                      .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
#                      .filter(ee.Filter.date(d1,d2))
#                      .filterBounds(roi_small))
#     s1_collection = s1_collection.map(speckle)


    water = dw_collection.select('water').mean().clip(clipping_mask).rename('water');
    label = dw_collection.select('label').mode().clip(clipping_mask).rename('label');
    flooded_vegetation = dw_collection.select('flooded_vegetation').mean().clip(clipping_mask).rename('flooded_vegetation');
    bare = dw_collection.select('bare').mean().clip(clipping_mask).rename('bare');
    trees = dw_collection.select('trees').mean().clip(clipping_mask).rename('trees');
    grass = dw_collection.select('grass').mean().clip(clipping_mask).rename('grass');
    crops = dw_collection.select('crops').mean().clip(clipping_mask).rename('crops');
    shrub_and_scrub = dw_collection.select('shrub_and_scrub').mean().clip(clipping_mask).rename('shrub_and_scrub');
    built = dw_collection.select('built').mean().clip(clipping_mask).rename('built');
    snow_and_ice = dw_collection.select('snow_and_ice').mean().clip(clipping_mask).rename('snow_and_ice');
    dw_img = water.addBands(ee.Image([bare,flooded_vegetation,trees,grass,crops,shrub_and_scrub,built,snow_and_ice]))


#     # s2
#     B2 = s2_collection.select('B2').mean().clip(clipping_mask).rename('B2').divide(10000);
#     B3 = s2_collection.select('B3').mean().clip(clipping_mask).rename('B3').divide(10000);
#     B4 = s2_collection.select('B4').mean().clip(clipping_mask).rename('B4').divide(10000);
#     B8 = s2_collection.select('B8').mean().clip(clipping_mask).rename('B8').divide(10000);
#     B11 = s2_collection.select('B11').mean().clip(clipping_mask).rename('B11').divide(10000); # SWIR1
#     B12 = s2_collection.select('B12').mean().clip(clipping_mask).rename('B12').divide(10000); # SWIR2
#     s2_img = B2.addBands(ee.Image([B3,B4,B8,B11,B12]))


#     # s1
#     s1_img = s1_collection.select(['VV_filtered','VH_filtered','VV_VH_filtered'])




    txt = d1 + '_' + d2


    # export dynamic world data
    txt2 = 'dynamic_world_small_bands_' + txt
    task = ee.batch.Export.image.toDrive(image=dw_img,
                                         description=txt2,
                                         scale=10,
                                         fileNamePrefix=txt2,
                                         crs='EPSG:4326',
                                         fileFormat='GeoTIFF',
                                         maxPixels=2e10,
                                         region=clipping_mask.geometry())
    task.start()
    task_id.append(task.id)


    txt2 = 'dynamic_world_small_label_' + txt
    task = ee.batch.Export.image.toDrive(image=label,
                                         description=txt2,
                                         scale=10,
                                         fileNamePrefix=txt2,
                                         crs='EPSG:4326',
                                         fileFormat='GeoTIFF',
                                         maxPixels=2e10,
                                         region=clipping_mask.geometry())
    task.start()
    task_id.append(task.id)


#     # export s2 data
#     txt2 = 's2_small_' + txt
#     task = ee.batch.Export.image.toDrive(image=s2_img,
#                                          description=txt2,
#                                          scale=10,
#                                          fileNamePrefix=txt2,
#                                          crs='EPSG:4326',
#                                          fileFormat='GeoTIFF',
#                                          maxPixels=2e10,
#                                          region=clipping_mask.geometry())
#     task.start()
#     task_id.append(task.id)


#     # export s1 data
#     txt2 = 's1_' + txt
#     task = ee.batch.Export.image.toDrive(image=s1_img.first(),
#                                          description=txt2,
#                                          scale=100,
#                                          fileNamePrefix=txt2,
#                                          crs='EPSG:4326',
#                                          fileFormat='GeoTIFF',
#                                          maxPixels=2e10,
#                                          region=s1_boundary.geometry())
#     task.start()
#     task_id.append(task.id)
    
    print(d1, ' ', d2)    

2020-01-15   2020-01-17
2020-01-26   2020-01-29
2020-02-07   2020-02-11
2020-02-20   2020-02-23
2020-03-01   2020-03-05
2020-04-07   2020-04-14
2020-04-30   2020-05-03
2020-05-12   2020-05-16
2020-05-25   2020-05-28
2020-09-22   2020-09-25
2020-10-27   2020-10-31
2020-11-08   2020-11-12
2020-11-21   2020-11-25
2020-12-03   2020-12-07


In [15]:
task.list()

[<Task FTD4SBOAWONNMBXCIFI7ZWAN EXPORT_IMAGE: dynamic_world_small_label_2020-03-25_2020-03-30 (RUNNING)>,
 <Task BHVTRV7P64U5L6JD6JVDTN6H EXPORT_IMAGE: dynamic_world_small_label_2020-03-25_2020-03-30 (COMPLETED)>,
 <Task TRHMLW5VW3X3FAOTPK5XANSJ EXPORT_IMAGE: dynamic_world_small_bands_2020-03-25_2020-03-30 (COMPLETED)>,
 <Task OP3ELE5EQU7KZVWBDJYLG4BU EXPORT_IMAGE: dynamic_world_small_label_2020-12-03_2020-12-07 (COMPLETED)>,
 <Task TIH3BWWD2YP42PKANGOP5LNQ EXPORT_IMAGE: dynamic_world_small_bands_2020-12-03_2020-12-07 (COMPLETED)>,
 <Task GX2DV6ODCI7BOGCJCRWZS3SB EXPORT_IMAGE: dynamic_world_small_label_2020-11-21_2020-11-25 (COMPLETED)>,
 <Task VTZFOTOTSUIIH3MROSI3FV5X EXPORT_IMAGE: dynamic_world_small_bands_2020-11-21_2020-11-25 (COMPLETED)>,
 <Task IYJW34GW4UORGUHOONYA22KC EXPORT_IMAGE: dynamic_world_small_label_2020-11-08_2020-11-12 (COMPLETED)>,
 <Task YEWIYE2DERU6RMNNONZE3J5T EXPORT_IMAGE: dynamic_world_small_bands_2020-11-08_2020-11-12 (COMPLETED)>,
 <Task 4ZRXT5UTEN62NY7L5QJ42DW

In [13]:
# get data for specific date


d1 = '2020-03-25'
d2 = '2020-03-30'

# get Dynamic World data
dw_collection = (ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")
    .filterBounds(s1_boundary)
    .filterDate(d1, d2))    


# print number of images in collection
print(dw_collection.size().getInfo())

# print dates of acquisition
acq_times = dw_collection.aggregate_array('system:time_start').getInfo()
print([time.strftime('%m-%d-%Y', time.gmtime(t/1000)) for t in acq_times])


#     water = dw_collection.select('water').mean().clip(clipping_mask).rename('water');
label = dw_collection.select('label').mode().clip(clipping_mask).rename('label');
#     flooded_vegetation = dw_collection.select('flooded_vegetation').mean().clip(clipping_mask).rename('flooded_vegetation');
#     bare = dw_collection.select('bare').mean().clip(clipping_mask).rename('bare');
#     trees = dw_collection.select('trees').mean().clip(clipping_mask).rename('trees');
#     grass = dw_collection.select('grass').mean().clip(clipping_mask).rename('grass');
#     crops = dw_collection.select('crops').mean().clip(clipping_mask).rename('crops');
#     shrub_and_scrub = dw_collection.select('shrub_and_scrub').mean().clip(clipping_mask).rename('shrub_and_scrub');
#     built = dw_collection.select('built').mean().clip(clipping_mask).rename('built');
#     snow_and_ice = dw_collection.select('snow_and_ice').mean().clip(clipping_mask).rename('snow_and_ice');
#     dw_img = water.addBands(ee.Image([bare,flooded_vegetation,trees,grass,crops,shrub_and_scrub,built,snow_and_ice]))


task_id = []
txt = d1 + '_' + d2


#     # export dynamic world data
#     txt2 = 'dynamic_world_small_bands_' + txt
#     task = ee.batch.Export.image.toDrive(image=dw_img,
#                                          description=txt2,
#                                          scale=10,
#                                          fileNamePrefix=txt2,
#                                          crs='EPSG:4326',
#                                          fileFormat='GeoTIFF',
#                                          maxPixels=2e10,
#                                          region=clipping_mask.geometry())
#     task.start()
#     task_id.append(task.id)


txt2 = 'dynamic_world_small_label_' + txt
task = ee.batch.Export.image.toDrive(image=label,
                                     description=txt2,
                                     scale=10,
                                     fileNamePrefix=txt2,
                                     crs='EPSG:4326',
                                     fileFormat='GeoTIFF',
                                     maxPixels=2e10,
                                     region=clipping_mask.geometry())
task.start()
task_id.append(task.id)


print(d1, ' ', d2)    

9
['03-28-2020', '03-28-2020', '03-28-2020', '03-28-2020', '03-28-2020', '03-28-2020', '03-28-2020', '03-28-2020', '03-28-2020']
2020-03-25   2020-03-30
